<img src="./01/logo-insa.png" style="float:right; max-width: 60px; display: inline" alt="SizingLab" /></a>

# Example of a generic sizing code
*Written by Marc Budinger (INSA Toulouse) and Scott Delbecq (ISAE-SUPAERO), Toulouse, France.*

### Main problems to be solved

In general, the establishment of a sizing procedure involves the resolution of the following 3 problems:
* a set of equations sub-constrained by the addition of a design variable in the optimization problem;
* an over-constrained variable by adding a design variable (multiplier) and the transfer of the excess equation(s) in the constrained part of the optimization problem;
* an algebraic loop by the use of a simplified equation weighted by a multiplying coefficient and a constraint representing the initial equation.


### Caution for the optimizer

Where possible:
* the design variables must take the form of a normalized variable around 1 (oversize coef. for example) or easily bounded to facilitate the work of the optimizer.
* the constraints must take the form of inequality and not of equality which often introduce numerical difficulties. The optimization of the objective (for example the total mass) will certainly force some (active) constraints to come to an end. 

### Example of generic sizing code

Below, you will find an example of an optimization code to solve the following problem.  

>For a tank of $x,y,z$ dimensions with $x\in[0.1,1]$, $y\in[0.1,1]$ and $z\in[0.1,2.5]$ m:   
Minimize the surface $xy+2z(x+y)$ 
and 
respect the constraints      
 - volume $xyz$ bigger than 200 l.  
 - developped dimensions $2z+x$ and $2z+y$ lower than 1.5m for machining feasability.  
   
First work is to define a `SizingCode` function which enables:
- to calculate the objective to minimize
- to calculate the constraints
- to print the results


In [4]:
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interactive

pd.options.display.float_format = '{:,.2f}'.format


# -----------------------
# sizing code
# -----------------------
# inputs: 
# - param: optimisation variables vector (reduction ratio, oversizing coefficient)
# - arg: selection of output  
# output: 
# - objective if arg='Obj', problem characteristics if arg='Prt', constraints other else

def SizingCode(param, arg):
# Design variables in param
# ---
    x = param[0] # variable x
    y = param[1] # variable y 
    z = param[2] # variable z
    
# Calculus 
# ---
    volume = x*y*z
    surface = x*y+2*(x+y)*z
        
# Objective and Constraints sum up
# ---
    objective = surface
    
    constraints = [volume
                   -.2,
                   1.5-(2*z+x),
                   1.5-(2*z+y) ]
    
# Returns selection
# -------------------
    if arg == 'Obj':
        return objective
    
    elif arg == 'ObjP':
        P = 0. # Penalisation nulle
        for C in constraints: 
            if (C < 0.): 
                P = P-1e9*C
        return objective + P # Return penalized objective 


    elif arg=='Prt':
    # the data to print a defined into a Pandas dataframe
        col_names = ['Type', 'Name', 'Value', 'Unit', 'Comment']

        df = pd.DataFrame()
    
      
        df = df.append([{'Type': 'Objective', 'Name': 'Surface', 'Value': objective, 'Unit': '[m^2]', 'Comment': 'Mini surface'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 1', 'Value': constraints[0], 'Unit': '[m^3]', 'Comment': 'Volume'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 2', 'Value': constraints[1], 'Unit': '[m]', 'Comment': 'Developped length'}])[col_names]
        df = df.append([{'Type': 'Constraints', 'Name': 'Const 3', 'Value': constraints[2], 'Unit': '[m]', 'Comment': 'Developped length'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'x', 'Value': x, 'Unit': '[m]', 'Comment': 'x'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'y', 'Value': y, 'Unit': '[m]', 'Comment': 'y'}])[col_names]
        df = df.append([{'Type': 'Variables', 'Name': 'z', 'Value': z, 'Unit': '[m]', 'Comment': 'z'}])[col_names]
        df = df.append([{'Type': 'Tank', 'Name': 'Volume', 'Value': volume, 'Unit': '[m^3]', 'Comment': 'Tank volume'}])[col_names]
        df = df.append([{'Type': 'Tank', 'Name': 'Surface', 'Value': surface, 'Unit': '[m^2]', 'Comment': 'Tank surface'}])[col_names]
        df = df.append([{'Type': 'Tank', 'Name': 'Developped length', 'Value': 2*z+x, 'Unit': '[m]', 'Comment': 'x length'}])[col_names]        
        df = df.append([{'Type': 'Tank', 'Name': 'Developped length', 'Value': 2*z+y, 'Unit': '[m]', 'Comment': 'y length'}])[col_names]

    # the dataframe is then organised to be printed with interactive widgets
        items = sorted(df['Type'].unique().tolist())

        def f(Type):
            return df[df['Type']==Type] 
        widgets.interact(f, Type=items)
        return f
        
    else:
        return constraints

### Optimization algorithms

We will now use the [optimization algorithms](https://docs.scipy.org/doc/scipy/reference/optimize.html) of the Scipy package to solve and optimize the configuration. We use here the SLSQP algorithm without explicit expression of the gradient (Jacobian). For global evaluation of the solution, you can use the differential evolution algorithm.


In [7]:
import scipy
import scipy.optimize

# Optimization algorithm choice
SLSQP = True

# Vector of initial parameters
parameters = scipy.array((1,1,1))

# Optimization with SLSQP algorithm: definition of requested functions
contrainte = lambda x: SizingCode(x, 'Const')
objectif = lambda x: SizingCode(x, 'Obj')

# Optimization with differential evolution algorithm
objectifP = lambda x: SizingCode(x, 'ObjP')

# Optimization bounds
bounds = [(.1,1), (.1,1), (0.1,2.5)]

# Optimisation
if SLSQP == True:
    # SLSQP optimisation
    result = scipy.optimize.fmin_slsqp(func=objectif, x0=parameters, bounds=bounds,f_ieqcons=contrainte, iter=1500, acc=1e-12)
    
else:
    # Differential evolution omptimisation
    result = scipy.optimize.differential_evolution(func=objectifP,
                                   bounds=bounds,
                                   tol=1e-12)


# Final characteristics after optimization 
print("-----------------------------------------------")
print("Final characteristics after optimization :")

if SLSQP == True:
    #SizingCode(result,'Obj')
    SizingCode(result, 'Prt')
else:
    #SizingCode(result.x,'Obj')
    SizingCode(result.x, 'Prt')
print("-----------------------------------------------")


,Type,Name,Value,Unit,Comment
0,Constraints,Const 1,-0.00,[m^3],Volume
0,Constraints,Const 2,0.03,[m],Developped length
0,Constraints,Const 3,0.03,[m],Developped length


-----------------------------------------------
